In [7]:
import requests
import pandas as pd
import numpy as np
from google.cloud import storage

def fetch_data():
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    querystring = {"team": "50", "league": "39", "season": "2022"}
    headers = {
        "X-RapidAPI-Key": "b01fb7d4d5msh9d6010d034fce6bp136078jsnfe71045ac5ee",
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    return response.json()

def process_data(data):
    players_data = []
    for player in data['response']:
        player_info = player['player']
        stats = player['statistics'][0]
        player_data = {
            "Player name": player_info['name'],
            "Nationality": player_info['nationality'],
            "Position": player_info.get('position', 'N/A'),
            "Weight": player_info.get('weight', 'N/A'),
            "Height": player_info.get('height', 'N/A'),
            "Team": stats['team']['name'],
            "League": stats['league']['name'],
            "Appearances": stats['games']['appearences'],
            "Lineups": stats['games']['lineups'],
            "Minutes": stats['games']['minutes'],
            "Goals": stats['goals']['total'],
            "Assists": stats['goals']['assists'],
            "Yellow Cards": stats['cards']['yellow'],
            "Red Cards": stats['cards']['red'],
            "Shots Total": stats['shots']['total'],
            "Shots On": stats['shots']['on'],
            "Passes Total": stats['passes']['total'],
            "Key Passes": stats['passes']['key'],
            "Pass Accuracy": stats['passes']['accuracy'],
            "Tackles Total": stats['tackles']['total'],
            "Blocks": stats['tackles']['blocks'],
            "Interceptions": stats['tackles']['interceptions'],
            "Dribbles Attempts": stats['dribbles']['attempts'],
            "Dribbles Success": stats['dribbles']['success'],
            "Dribbled Past": stats['dribbles']['past'],
            "Fouls Drawn": stats['fouls']['drawn'],
            "Fouls Committed": stats['fouls']['committed'],
            "Duels Total": stats['duels']['total'],
            "Duels Won": stats['duels']['won'],
            "Penalty Scored": stats['penalty']['scored'],
            "Penalty Missed": stats['penalty']['missed'],
            "Penalty Saved": stats['penalty']['saved']
        }
        players_data.append(player_data)
    return pd.DataFrame(players_data)

def clean_and_transform_data(df):

    # Drop unnecessary columns
    df = df.drop(['Position', 'Dribbled Past'], axis=1)

    # Cleaning Weight column
    # Remove 'kg' and convert 'none' to NaN, then convert to numeric
    df['Weight'] = df['Weight'].str.replace(' kg', '').str.replace('kg', '')
    df['Weight'].replace('none', np.nan, inplace=True)
    df['Weight'] = pd.to_numeric(df['Weight'], errors='coerce')

    # Calculate the average weight and replace NaN values with it
    average_weight = df['Weight'].dropna().mean()
    df['Weight'].fillna(average_weight, inplace=True)

    # Cleaning Height column
    # Remove 'cm' and convert 'none' to NaN, then convert to numeric
    df['Height'] = df['Height'].str.replace(' cm', '').str.replace('cm', '')
    df['Height'].replace('none', np.nan, inplace=True)
    df['Height'] = pd.to_numeric(df['Height'], errors='coerce')

    # Calculate the average height and replace NaN values with it
    average_height = df['Height'].dropna().mean()
    df['Height'].fillna(average_height, inplace=True)

    # Replace all other NaN values in the DataFrame with 0
    df.fillna(0, inplace=True)
    
    # Remove the first row
    df = df.iloc[1:]

    return df

def upload_to_gcs(df, bucket_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Save DataFrame to a CSV on memory
    df.to_csv(destination_blob_name, index=False)

    # Upload the CSV to GCS
    blob.upload_from_filename(destination_blob_name)
    print(f"File {destination_blob_name} uploaded to {bucket_name}.")



In [8]:
data = fetch_data()
df = process_data(data)
df

,Player name,Nationality,Position,Weight,Height,Team,League,Appearances,Lineups,Minutes,...,Dribbles Attempts,Dribbles Success,Dribbled Past,Fouls Drawn,Fouls Committed,Duels Total,Duels Won,Penalty Scored,Penalty Missed,Penalty Saved
0,B. Mendy,France,N/A,85 kg,185 cm,Manchester City,Premier League,NaN,NaN,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
1,F. Burns,England,N/A,None,185 cm,Manchester City,Premier League,NaN,NaN,NaN,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
2,N. O&apos;Reilly,England,N/A,None,None,Manchester City,Premier League,0.0,0.0,0.0,...,NaN,NaN,None,NaN,NaN,NaN,NaN,0.0,0.0,None
3,B. Knight,England,N/A,61 kg,171 cm,Manchester City,Premier League,0.0,0.0,0.0,...,NaN,NaN,None,NaN,NaN,NaN,NaN,0.0,0.0,None
4,A. Robertson,Australia,N/A,None,182 cm,Manchester City,Premier League,0.0,0.0,0.0,...,NaN,NaN,None,NaN,NaN,NaN,NaN,0.0,0.0,None
5,L. Mbete,England,N/A,82 kg,185 cm,Manchester City,Premier League,0.0,0.0,0.0,...,NaN,NaN,None,NaN,NaN,NaN,NaN,0.0,0.0,None
6,J. Grealish,England,N/A,76 kg,175 cm,Manchester City,Premier League,28.0,23.0,2062.0,...,73.0,45.0,None,78.0,27.0,270.0,162.0,0.0,0.0,None
7,İ. Gündoğan,Germany,N/A,80 kg,180 cm,Manchester City,Premier League,31.0,27.0,2359.0,...,35.0,16.0,None,25.0,23.0,192.0,90.0,0.0,1.0,None
8,C. Palmer,England,N/A,74 kg,182 cm,Manchester City,Premier League,14.0,2.0,358.0,...,21.0,14.0,None,6.0,1.0,47.0,26.0,0.0,0.0,None
9,J. Álvarez,Argentina,N/A,71 kg,170 cm,Manchester City,Premier League,31.0,13.0,1451.0,...,35.0,21.0,None,14.0,21.0,170.0,69.0,1.0,0.0,None


In [9]:
df_cleaned = clean_and_transform_data(df)
df_cleaned

,Player name,Nationality,Weight,Height,Team,League,Appearances,Lineups,Minutes,Goals,...,Interceptions,Dribbles Attempts,Dribbles Success,Fouls Drawn,Fouls Committed,Duels Total,Duels Won,Penalty Scored,Penalty Missed,Penalty Saved
1,F. Burns,England,75.4375,185.000000,Manchester City,Premier League,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,N. O&apos;Reilly,England,75.4375,179.368421,Manchester City,Premier League,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,B. Knight,England,61.0000,171.000000,Manchester City,Premier League,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,A. Robertson,Australia,75.4375,182.000000,Manchester City,Premier League,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,L. Mbete,England,82.0000,185.000000,Manchester City,Premier League,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,J. Grealish,England,76.0000,175.000000,Manchester City,Premier League,28.0,23.0,2062.0,5.0,...,7.0,73.0,45.0,78.0,27.0,270.0,162.0,0.0,0.0,0
7,İ. Gündoğan,Germany,80.0000,180.000000,Manchester City,Premier League,31.0,27.0,2359.0,8.0,...,20.0,35.0,16.0,25.0,23.0,192.0,90.0,0.0,1.0,0
8,C. Palmer,England,74.0000,182.000000,Manchester City,Premier League,14.0,2.0,358.0,0.0,...,1.0,21.0,14.0,6.0,1.0,47.0,26.0,0.0,0.0,0
9,J. Álvarez,Argentina,71.0000,170.000000,Manchester City,Premier League,31.0,13.0,1451.0,9.0,...,2.0,35.0,21.0,14.0,21.0,170.0,69.0,1.0,0.0,0
10,E. Haaland,Norway,88.0000,194.000000,Manchester City,Premier League,35.0,33.0,2779.0,36.0,...,3.0,29.0,11.0,24.0,31.0,217.0,88.0,7.0,0.0,0
